### 1) Загрузка таблиц по ссылкам и объединение ###

In [35]:
import pandas as pd
import numpy as np
from IPython.display import display

url_1 = "https://pstu.ru/files/file/Abitur/2025%20final/%D0%93%D0%9D%D0%A4_%D0%91_%D0%9D_000006989.html"
url_2 = "https://pstu.ru/files/file/Abitur/2025%20final/%D0%93%D0%9D%D0%A4_%D0%91_%D0%98_000007463.html"


def read_table(url: str) -> pd.DataFrame:
    tables = pd.read_html(url, header=0, skiprows=12)
    df = tables[0].copy()
    return df

# Загружаем обе таблицы и объединяем
top = read_table(url_1)
bottom = read_table(url_2)
raw_df = pd.concat([top, bottom], ignore_index=True, sort=False)
display(raw_df)


,№,Уникальный код,Сумма баллов,АиНМА / Математика,ПФ / География / Информатика и ИКТ / Физика / Химия,Русский язык,Сумма баллов за ИД,Учебная группа,Представление приказа,Идентификационный номер заказчика целевого обучения (для целевого приема),Номер предложения (для целевого приема),Unnamed: 11,Unnamed: 12
0,1,4951473,258,82,88,88,0,ФП -25-1с,№ 07-08/2-С от 07.08.2025,NaN,NaN,NaN,NaN
1,2,4951303,258,82,88,88,0,ФП -25-1с,№ 07-08/2-С от 07.08.2025,NaN,NaN,NaN,NaN
2,3,4951155,254,88,78,88,0,ФП -25-1с,№ 07-08/2-С от 07.08.2025,NaN,NaN,NaN,NaN
3,4,4950639,252,88,82,82,0,ФП -25-1с,№ 07-08/2-С от 07.08.2025,NaN,NaN,NaN,NaN
4,5,4951362,252,82,82,88,0,ФП -25-1с,№ 07-08/2-С от 07.08.2025,NaN,NaN,NaN,NaN
5,6,3653144,252,82,82,88,0,ФП -25-1с,№ 07-08/2-С от 07.08.2025,NaN,NaN,NaN,NaN
6,7,4949693,248,82,78,88,0,ФП -25-1с,№ 07-08/2-С от 07.08.2025,NaN,NaN,NaN,NaN
7,8,4953071,247,82,100,65,0,ФП -25-1с,№ 07-08/2-С от 07.08.2025,NaN,NaN,NaN,NaN
8,9,4151976,238,76,90,72,0,ФП -25-1с,№ 07-08/2-С от 07.08.2025,NaN,NaN,NaN,NaN
9,10,3948500,226,74,74,78,0,ФП -25-1с,№ 07-08/2-С от 07.08.2025,NaN,NaN,NaN,NaN


### 2) Приведение к "рабочему" виду ###

In [ ]:
df = raw_df.copy()

# Заменяем сложные названия предметов на простые
df.columns = [str(c).strip().replace("\n", " ") for c in df.columns]
rename_map = {}
for c in df.columns:
    if ('АиНМА' in c) or ('Математика' in c):
        rename_map[c] = 'Предмет №1'
    elif any(k in c for k in ['ПФ', 'География', 'Информатика и ИКТ', 'Физика', 'Химия']):
        rename_map[c] = 'Предмет №2'
    elif 'Русский' in c:
        rename_map[c] = 'Русский язык'
df = df.rename(columns=rename_map)

# Удаляем полностью пустые столбцы
df = df.dropna(axis=1, how='all')

# Нормализуем заголовки
df.columns = [str(c).strip().replace('\n', ' ') for c in df.columns]

# Удаляем служебные столбцы, если присутствуют
cols_to_drop = [
    '№',
    'Уникальный код',
    'Представление приказа',
    'Учебная группа',
    'Идентификационный номер заказчика целевого обучения (для целевого приема)',
    'Номер предложения (для целевого приема)'
]
df = df.drop(columns=[c for c in cols_to_drop if c in df.columns], errors='ignore')

# Переустанавливаем индексацию
df = df.reset_index(drop=True)
df.index = df.index + 1
df.index.name = '№'

display(df.head(15))


,Сумма баллов,Предмет №1,Предмет №2,Русский язык,Сумма баллов за ИД
№,,,,,
1,258,82,88,88,0
2,258,82,88,88,0
3,254,88,78,88,0
4,252,88,82,82,0
5,252,82,82,88,0
6,252,82,82,88,0
7,248,82,78,88,0
8,247,82,100,65,0
9,238,76,90,72,0


### 3) Проверка типов и приведение количественных к числовому типу ###

In [20]:
print("До преобразования:")
print(df.dtypes)

numeric_candidates = [
    c for c in df.columns
    if any(k in c for k in ['Сумма баллов', 'Предмет №1', 'Предмет №2', 'Русский'])
]

for c in numeric_candidates:
    df[c] = pd.to_numeric(
        df[c].astype(str).str.replace(',', '.', regex=False),
        errors='coerce'
    )

print("\nПосле преобразования:")
print(df[numeric_candidates].dtypes)

display(df)


До преобразования:
Сумма баллов          int64
Предмет №1            int64
Предмет №2            int64
Русский язык          int64
Сумма баллов за ИД    int64
dtype: object

После преобразования:
Сумма баллов          int64
Предмет №1            int64
Предмет №2            int64
Русский язык          int64
Сумма баллов за ИД    int64
dtype: object


,Сумма баллов,Предмет №1,Предмет №2,Русский язык,Сумма баллов за ИД
№,,,,,
1,258,82,88,88,0
2,258,82,88,88,0
3,254,88,78,88,0
4,252,88,82,82,0
5,252,82,82,88,0
6,252,82,82,88,0
7,248,82,78,88,0
8,247,82,100,65,0
9,238,76,90,72,0


### 4) Средний, минимальный и максимальный суммарные баллы ###

In [21]:
if 'Сумма баллов' not in df.columns:
    raise ValueError("Колонка 'Сумма баллов' не найдена.")

mean_total = df['Сумма баллов'].mean()
min_total = df['Сумма баллов'].min()
max_total = df['Сумма баллов'].max()

print(f"Средняя сумма баллов: {mean_total:.2f}")
print("Минимальная сумма баллов:", min_total)
print("Максимальная сумма баллов:", max_total)
display(df)


Средняя сумма баллов: 232.27
Минимальная сумма баллов: 154
Максимальная сумма баллов: 258


,Сумма баллов,Предмет №1,Предмет №2,Русский язык,Сумма баллов за ИД
№,,,,,
1,258,82,88,88,0
2,258,82,88,88,0
3,254,88,78,88,0
4,252,88,82,82,0
5,252,82,82,88,0
6,252,82,82,88,0
7,248,82,78,88,0
8,247,82,100,65,0
9,238,76,90,72,0


### 5) Средние по трём предметам и предмет с наибольшим средним ###

In [22]:
required_cols = ['Предмет №1', 'Предмет №2', 'Русский язык']
missing = [c for c in required_cols if c not in df.columns]

means_df = pd.DataFrame({
    'Предмет №1': [df['Предмет №1'].mean()],
    'Предмет №2': [df['Предмет №2'].mean()],
    'Русский язык': [df['Русский язык'].mean()],
}, index=['Среднее'])

display(means_df)

top_subject = means_df.idxmax(axis=1).values[0]
print("Предмет с наибольшим средним баллом:", top_subject)


,Предмет №1,Предмет №2,Русский язык
Среднее,78.0,78.133333,76.133333


Предмет с наибольшим средним баллом: Предмет №2


### 6) Количество абитуриентов Тип_1 ###

In [38]:
candidates = {c: c for c in df.columns if c in ['Русский язык', 'Предмет №1', 'Предмет №2']}
rus_col = candidates.get('Русский язык')
subj1_col = candidates.get('Предмет №1')
subj2_col = candidates.get('Предмет №2')

rus_mean = df[rus_col].mean() if rus_col else np.nan
subj1_mean = df[subj1_col].mean()
subj2_mean = df[subj2_col].mean()

if all([rus_col, subj1_col, subj2_col]):
    type_1 = int(((df[rus_col] < rus_mean) & (df[subj1_col] > subj1_mean) & (df[subj2_col] > subj2_mean)).sum())
else:
    type_1 = 0

print("Кол-во студентов Тип_1:", type_1)


Кол-во студентов Тип_1: 1


### 7) Количество абитуриентов Тип_2 ###

In [24]:
if all([rus_col, subj1_col, subj2_col]):
    type_2 = int(((df[rus_col] > rus_mean) & (df[subj1_col] < subj1_mean) & (df[subj2_col] < subj2_mean)).sum())
else:
    type_2 = 0

print("Кол-во студентов Тип_2:", type_2)


Кол-во студентов Тип_2: 1


### 8) Итоговый DataFrame с метаданными ###

In [43]:
summary_df = pd.DataFrame([
    {
        "Подразделение": "Горно-нефтяной факультет",
        "Уровень подготовки": "Специалитет",
        "Направление подготовки/специальность": "Физические процессы горного или нефтегазового производства",
        "Год": 2025,
        "Количество мест": int(len(df)),
        "Предметы ЕГЭ": "Предмет №1, Предмет №2, Русский язык",
        "Средняя сумма баллов": float(mean_total),
        "Min сумма баллов": float(min_total),
        "Max сумма баллов": float(max_total),
        "Предмет с высшим средним": top_subject,
        "Кол-во студентов тип_1": int(type_1),
        "Кол-во студентов тип_2": int(type_2),
    }
])

display(summary_df)


,Подразделение,Уровень подготовки,Направление подготовки/специальность,Год,Количество мест,Предметы ЕГЭ,Средняя сумма баллов,Min сумма баллов,Max сумма баллов,Предмет с высшим средним,Кол-во студентов тип_1,Кол-во студентов тип_2
0,Горно-нефтяной факультет,Специалитет,Физические процессы горного или нефтегазового ...,2025,15,"Предмет №1, Предмет №2, Русский язык",232.266667,154.0,258.0,Предмет №2,1,1
